# Query graph generator

This code generates query graphs in similar manner as described in the paper _[Adaptive Optimization of Very Large Join Queries](https://dl.acm.org/doi/pdf/10.1145/3183713.3183733)_ and in _[Heuristic and Randomized Optimization for the Join Ordering Problem](https://link.springer.com/content/pdf/10.1007/s007780050040.pdf)_. Relations in the database are simply modelled as positive integers which is the default for networkx. Besides the graph, we create cardinality distribution over the relations in the query. This distribution is given as a list of positive integers where the cardinality stored in the list index $i$, corresponds to the table $i$ in the graph.

TODO benchmarks: TPC-H, TPC-DS, LDBC, JOB, SQLite

## Synthetic workloads

We generate larger synthetic workloads. This generation process follows the idea from the paper _[Adaptive Optimization of Very Large Join Queries](https://dl.acm.org/doi/pdf/10.1145/3183713.3183733)_. Besides, chain, star, cycle and grid query graphs, we generate complete graphs which demonstrate the hardest case to optimize. Complete query graphs have not been considered in research as far as we know. 

In [1]:
import networkx as nx
from networkx.readwrite import json_graph
import json
import numpy as np
import itertools
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import os
notebook_path = os.path.abspath("query_graph_generator.ipynb")

In [2]:
def generate_query_graph(n, rel_size_scale, graph_type):
    query_graph = None
    if graph_type == "cycle":
        query_graph = nx.cycle_graph(n)
    elif graph_type == "chain":
        query_graph = nx.path_graph(n)
    elif graph_type == "grid":
        dim = int(np.floor(np.sqrt(n)))
        query_graph = nx.grid_2d_graph(dim, dim)
    elif graph_type == "complete":
        query_graph = nx.complete_graph(n)
    elif graph_type == "star":
        query_graph = nx.star_graph(n)
            
    rel_dist = np.random.random_sample(size=n)
    rel_dist = [int(np.ceil(value*rel_size_scale)) for value in rel_dist]
    return query_graph, rel_dist

Example:

In [3]:
G, distribution = generate_query_graph(4, 1000, "complete")
print("Cardinality distribution: ", distribution)
nx.draw(G, with_labels=True, font_weight='bold')

Cardinality distribution:  [78, 741, 409, 121]


In [4]:
graph_types = ["cycle", "chain", "grid", "complete", "star"]

for ty in graph_types:
    for n in range(2, 11):
        G, distribution = generate_query_graph(2**n, 1000000, ty)
        f = plt.figure(figsize=(10*n,10*n))
        nx.draw(G, with_labels=True, font_weight='bold')
        graph_image_path = os.path.join(os.path.dirname(notebook_path), "data/" + ty + "/" + str(n) + "/graph-" + str(n) + ".png")
        f.savefig(graph_image_path)
        json_file_path = os.path.join(os.path.dirname(notebook_path), "data/" + ty + "/" + str(n) + "/graph-" + str(n) + ".json")
        graph_data = json_graph.node_link_data(G)
        with open(json_file_path, 'w') as graph_file:
            json.dump({'name': ty + "-graph-" + str(n), 'cardinality_distribution': distribution, 'graph': graph_data}, graph_file)

C:\Users\valte\AppData\Local\Temp\ipykernel_11192\3845097025.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f = plt.figure(figsize=(10*n,10*n))
